Setup

In [ ]:
import itertools
import numpy as np
from numpy.random import *
import graph_tool.all as gt

global rng
rng = np.random.default_rng(seed=42)

n = 100000
d = 512
t = 1
k = 32
k_adj = 2.0
k_m = k * k_adj

r_approx = 5420

if n >= 10^5:
    p = d / n
else:
    p = d / (n - 1)

def add_fast_gnp_edges():
    num_edges = rng.binomial(n*(n-1)/2, p)
    sources = rng.integers(0, n, num_edges*2)
    targets = rng.integers(0, n, num_edges*2)
    mask = sources != targets # removes self-loops
    g.add_edge_list(np.column_stack((sources[mask], targets[mask])))

global g
g = gt.Graph(directed=True)
g.add_vertex(n)
add_fast_gnp_edges()

mode_T = g.new_vp("int")
mode_q = g.new_vp("int")
mode_f = g.new_vp("int")

mode_T.a = t
mode_q.a = 1
mode_f.a = 0

mode_w = g.new_ep("double")
mode_qq = g.new_ep("int")

#Initialize synapse weights to 0
mode_w.a = 0
mode_qq.a = 1

def fast_sum_weights(s_i):
    W = g.get_in_edges(s_i, [mode_w])[:,2]
    F = np.array(g.get_in_neighbors(s_i, [mode_f])[:,1], dtype=bool)
    return W[F].sum()

def _delta(s_i, w_i):
    if w_i > mode_T[s_i]:
        mode_f[s_i] = 1
        mode_q[s_i] = 2

def _lambda(s_i, w_i, s_ji, f_j):
    if f_j == 1:
        mode_qq[s_ji] = 2

def update_graph(one_step=True):
    C = []
    for s_i in g.iter_vertices():
        w_i = fast_sum_weights(s_i)
        _delta(s_i, w_i)
        if mode_q[s_i] == 2:
            C.append(s_i)
            if one_step:
                mode_f[s_i] = 0
        if not one_step:
            for s_ji in g.iter_in_edges(s_i):
                f_j = mode_f[s_ji[0]]
                _lambda(s_i, w_i, s_ji, f_j)
    return C



def SJOIN_one_step(A):
    scale = 1.7
    while True:
        for i in A:
            mode_f[i] = 1
            mode_w.a[g.get_out_edges(i, [g.edge_index])[:,2]] = (t / k_m) * scale
        B = update_graph(one_step=True)
        mode_f.a = 0
        mode_q.a = 1
        print("{:.2f} : {}".format(scale, len(B)))
        if len(B) > 0.9*(r_approx):
            for i in A:
                out_edges_i = g.get_out_edges(i, [g.edge_index])
                neighbors_not_B = np.setdiff1d(out_edges_i[:,1], B)
                edges_to_deactivate = out_edges_i[out_edges_i[:,1] == neighbors_not_B][:,2]
                mode_w.a[edges_to_deactivate] = 0
            return B
        scale = scale + 0.05





In [9]:
A = rng.choice(np.arange(0,n-1), size=r_approx)
B = SJOIN_one_step(A)

for i in A:
    out = g.get_out_edges(A[0], [g.edge_index])[:,2]
    w = mode_w.a[out]
    print(np.count_nonzero(w))

1.70 : 2251
1.75 : 3124
1.80 : 4311
1.85 : 5638


/tmp/ipykernel_2271/3565224509.py:95: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  edges_to_deactivate = out_edges_i[out_edges_i[:,1] == neighbors_not_B][:,2]


483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
483
